# GPT-SoVITS version2のサンプル利用コード
##利用方法
・基本的には全てのセルを実行することで利用できます。

・GPUを利用しましょう。（利用制限があるのでこまめにインスタンスは削除してください）

（変更する際は，「ランタイム」→「ランタイムのタイプを変更」からGPUを選択してください）

##注意点
・本ファイルの相対的な場所は変更しないでください。

・本ファイルの名前を変更しないでください。

変更する場合は，[1]セル目のファイル名も一緒に変更してください。

・[2]セル目の実行時にセッションの再起動を求められることがあります。

その場合はポップアップに従って再起動したあと，全てのセルを再度実行してください。

In [1]:
#Google Driveのフォルダをマウント（認証入る）
from google.colab import drive
drive.mount('/content/drive')

# カレントディレクトリを本ファイルが存在するディレクトリに変更する。
import glob
import os
pwd = os.path.dirname(glob.glob('/content/drive/MyDrive/TTS-test/colab_GPT-SoVITS_sample.ipynb', recursive=True)[0])
print(pwd)

%cd $pwd
!pwd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/TTS-test
/content/drive/MyDrive/TTS-test
/content/drive/MyDrive/TTS-test


In [2]:
!pip install -r requirements.txt

Ignoring onnxruntime: markers 'sys_platform == "darwin"' don't match your environment
Ignoring opencc: markers 'sys_platform != "linux"' don't match your environment


In [3]:
import sys
sys.path.append("GPT_SoVITS")

import argparse
import os
import soundfile as sf
import time

from tools.i18n.i18n import I18nAuto
from GPT_SoVITS.inference_webui import change_gpt_weights, change_sovits_weights, get_tts_wav


<All keys matched successfully>
Number of parameter: 77.61M


In [4]:
## 設定

gpt_model = "./GPT_SoVITS/pretrained_models/gsv-v2final-pretrained/s1bert25hz-5kh-longer-epoch=12-step=369668.ckpt"
sovits_model = "./GPT_SoVITS/pretrained_models/gsv-v2final-pretrained/s2G2333k.pth"

ref_audio_path = "inputs/audio.wav"
ref_text = """
イタリア旅行で彼は、いくつか景勝の地として有名な都市、例えば、ナポリやフィレンツェを訪れた。
"""
ref_language = "日文"

target_text = """
今日は、お兄ちゃんが大好きな深層学習の勉強をしようね！
え？強化学習の方が・・・好き？？
お兄ちゃんのバカ！
"""

target_language = "日文"
output_wav_path = "./outputs/output.wav"


In [5]:
i18n = I18nAuto()
# Change model weights
change_gpt_weights(gpt_path=gpt_model)
change_sovits_weights(sovits_path=sovits_model)

IMPORTANT: You are using gradio version 3.38.0, however version 4.29.0 is available, please upgrade.
--------
Number of parameter: 77.61M
<All keys matched successfully>


In [6]:
start = time.time()
synthesis_result = get_tts_wav(ref_wav_path=ref_audio_path,
                                prompt_text=ref_text,
                                prompt_language=i18n(ref_language),
                                text=target_text,
                                text_language=i18n(target_language), top_p=1, temperature=1,inp_refs = False)

"""
#inp_refs:= List
「オプション：複数の参考音声ファイルをドラッグ＆ドロップすることでアップロードし、それらの音色を平均的に融合します（同性を推奨）。この項目を記入しない場合、音色は左側の単一の参考音声ファイルによって制御されます。微調整モデルの場合、参考音声は全て微調整トレーニングセットの音色内であることを推奨しますが、ベースモデルは関係ありません。」
"""

result_list = list(synthesis_result)

print(f"Time taken: {time.time() - start}")

if result_list:
    last_sampling_rate, last_audio_data = result_list[-1]
    sf.write(output_wav_path, last_audio_data, last_sampling_rate)
    print(f"Audio saved to {output_wav_path}")



Actual Input Reference Text: イタリア旅行で彼は、いくつか景勝の地として有名な都市、例えば、ナポリやフィレンツェを訪れた。
Actual Input Target Text: 今日は、お兄ちゃんが大好きな深層学習の勉強をしようね！
え？強化学習の方が・・・好き？？
お兄ちゃんのバカ！
Actual Input Target Text (after sentence segmentation): 今日は、お兄ちゃんが大好きな深層学習の勉強をしようね！
え？強化学習の方が・・・好き？？
お兄ちゃんのバカ！
Actual Input Target Text (per sentence): 今日は、お兄ちゃんが大好きな深層学習の勉強をしようね！
Processed text from the frontend (per sentence): 今日は、お兄ちゃんが大好きな深層学習の勉強をしようね！


  8%|▊         | 115/1500 [00:01<00:19, 69.63it/s]


T2S Decoding EOS [214 -> 330]
Actual Input Target Text (per sentence): え？強化学習の方が・・・好き？？
Processed text from the frontend (per sentence): え？強化学習の方が・・・好き？？


  5%|▍         | 70/1500 [00:00<00:12, 112.77it/s]


T2S Decoding EOS [214 -> 285]
Actual Input Target Text (per sentence): お兄ちゃんのバカ！
Processed text from the frontend (per sentence): お兄ちゃんのバカ！


  2%|▏         | 27/1500 [00:00<00:12, 116.94it/s]


T2S Decoding EOS [214 -> 242]
2.085	0.114	2.541	2.517
Time taken: 7.2607221603393555
Audio saved to ./outputs/output.wav


In [7]:
from IPython.display import display, Audio
display(Audio(last_audio_data, rate=last_sampling_rate))